In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import acf
from sqlalchemy import create_engine
from scipy.stats import bartlett, jarque_bera, levene, normaltest

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houses = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


houses.head(10)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


#### Model 1

In [2]:
houses = houses[['saleprice','overallqual','grlivarea','garagecars','garagearea','totalbsmtsf','fullbath','yearbuilt','yearremodadd']]

features = list(houses.columns)
features.remove('saleprice')
print('features (len: {}): {}\ntarget: saleprice'.format(len(features),{i for i in features}))

X = houses[features]

Y = houses['saleprice']

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

features (len: 8): {'totalbsmtsf', 'grlivarea', 'garagearea', 'yearremodadd', 'overallqual', 'yearbuilt', 'garagecars', 'fullbath'}
target: saleprice


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.772
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     615.5
Date:                Sun, 05 Jan 2020   Prob (F-statistic):               0.00
Time:                        14:55:34   Log-Likelihood:                -17463.
No. Observations:                1460   AIC:                         3.494e+04
Df Residuals:                    1451   BIC:                         3.499e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.158e+06   1.28e+05     -9.030      0.000   -1.41e+06   -9.07e+05
overallqual   1.923e+04   1183.567     16.248      0.000    1.69e+04    2.16e+04
grlivarea       53.8303      2.996     17.968      0.000      47.953      59.707
garagecars    1.054e+04   3039.969      3.467      0.001    4576.313    1.65e+04
garagearea      16.8986     10.286      1.643      0.101      -3.278      37.075
totalbsmtsf     28.9796      2.885     10.044      0.000      23.320      34.639
fullbath     -6394.9604   2660.790     -2.403      0.016   -1.16e+04   -1175.555
yearbuilt      257.9006     50.164      5.141      0.000     159.499     356.302
yearremodadd   294.6293     63.752      4.621      0.000     169.573     419.685
==============================================================================
Omnibus:                      485.531   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            60951.563
Skew:                          -0.441   Prob(JB):                         0.00
Kurtosis:                      34.641   Cond. No.                     4.39e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.39e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Model 2: Add interaction area_per_car to Model 1 and remove salesprice, garagearea and garagecars

From previous assignments, we see that GarageCars and GarageArea are strongly correlated based on correlation matrix and therefore one of them should be removed. Let's include interaction and see whether that can help our model:

In [6]:
# add a new column, area_per_car which is the interaction between garagearea and garagecars
houses['area_per_car'] = houses['garagearea']/houses['garagecars']

# convert nan/inf to numerical
houses['area_per_car'] = houses['area_per_car'].apply(np.nan_to_num)

features = list(houses.columns)
features.remove('saleprice')
features.remove('garagecars')
features.remove('garagearea')
print('features (len: {}): {}\ntarget: saleprice'.format(len(features),{i for i in features}))


X = houses[features]

Y = houses['saleprice']

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

features (len: 7): {'area_per_car', 'totalbsmtsf', 'grlivarea', 'yearremodadd', 'overallqual', 'yearbuilt', 'fullbath'}
target: saleprice


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     664.1
Date:                Sun, 05 Jan 2020   Prob (F-statistic):               0.00
Time:                        14:56:39   Log-Likelihood:                -17496.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1452   BIC:                         3.505e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.389e+06   1.28e+05    -10.849      0.000   -1.64e+06   -1.14e+06
overallqual   2.098e+04   1188.432     17.650      0.000    1.86e+04    2.33e+04
grlivarea       58.3438      3.007     19.403      0.000      52.445      64.242
totalbsmtsf     31.5994      2.896     10.913      0.000      25.919      37.280
fullbath     -5758.5222   2725.604     -2.113      0.035   -1.11e+04    -411.980
yearbuilt      365.8860     49.505      7.391      0.000     268.776     462.996
yearremodadd   304.6617     65.364      4.661      0.000     176.444     432.880
area_per_car    12.0881     11.942      1.012      0.312     -11.338      35.514
==============================================================================
Omnibus:                      560.039   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            70190.282
Skew:                          -0.737   Prob(JB):                         0.00
Kurtosis:                      36.936   Cond. No.                     4.26e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.26e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Model 3: Remove one feature garagearea from Model 1

That did not help the model. We will remove area_p_car and put back GarageCars because it is a -very small- bit more correlated with the target:

In [9]:
houses.drop(columns=["area_per_car", "garagearea"],inplace=True)

In [11]:
features = list(houses.columns)
features.remove('saleprice')
print('features (len: {}): {}\ntarget: saleprice'.format(len(features),{i for i in features}))

X = houses[features]

Y = houses['saleprice']

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

features (len: 7): {'totalbsmtsf', 'grlivarea', 'yearremodadd', 'overallqual', 'yearbuilt', 'garagecars', 'fullbath'}
target: saleprice


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.772
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     702.3
Date:                Sun, 05 Jan 2020   Prob (F-statistic):               0.00
Time:                        15:00:55   Log-Likelihood:                -17465.
No. Observations:                1460   AIC:                         3.495e+04
Df Residuals:                    1452   BIC:                         3.499e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         -1.16e+06   1.28e+05     -9.036      0.000   -1.41e+06   -9.08e+05
overallqual    1.92e+04   1184.098     16.214      0.000    1.69e+04    2.15e+04
grlivarea       54.3616      2.980     18.241      0.000      48.516      60.207
garagecars    1.455e+04   1810.225      8.039      0.000     1.1e+04    1.81e+04
totalbsmtsf     29.9299      2.828     10.582      0.000      24.382      35.478
fullbath     -6839.0505   2648.573     -2.582      0.010    -1.2e+04   -1643.613
yearbuilt      259.5217     50.184      5.171      0.000     161.081     357.962
yearremodadd   293.7672     63.787      4.605      0.000     168.642     418.892
==============================================================================
Omnibus:                      467.236   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            55591.705
Skew:                          -0.374   Prob(JB):                         0.00
Kurtosis:                      33.220   Cond. No.                     4.35e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.35e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

With one less feature we were able to get back to the best evaluation metrics; those of the original model. Moreover, the F-statistic is higher, meaning it is better than both the models.

Model 1
- F-statistic of 615.5 with a p-value of 0 means that our model does add to an empty model.

- R2 and Adjusted R-squared explains about 77% of variance in the target salesprice. 

- Higher AIC indicates overfitting and higher BIC value iindicates it may have penalized the model for excessive features or parameters.

- This gives an indication that additional feature engineering may be needed for our model.

Model 2
- F-statistic of 664.1 with a p-value of 0 means that our model does add to an empty model.

- R2 and Adjusted R-squared explains about 76% of variance in the target salesprice. 

Model 3
- F-statistic of 702.3 with a p-value of 0 means that our model does add to an empty model.

- R2 and Adjusted R-squared explains about 77% of variance in the target salesprice. 
